In [4]:
!pip install Flask pyngrok


from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
from pyngrok import ngrok

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///iam.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# User Model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password_hash = db.Column(db.String(120), nullable=False)
    role = db.Column(db.String(50), nullable=False)  # e.g., 'admin', 'user'

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

# Create Database
with app.app_context():
    db.create_all()

# Authentication Middleware
def authenticate(username, password):
    user = User.query.filter_by(username=username).first()
    if user and user.check_password(password):
        return user
    return None

# Authorization Middleware
def authorize(user, required_role):
    return user.role == required_role

# Routes
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    role = data.get('role', 'user')  # Default role is 'user'

    if User.query.filter_by(username=username).first():
        return jsonify({"message": "User already exists"}), 400

    user = User(username=username, role=role)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()
    return jsonify({"message": "User registered successfully"}), 201

@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    user = authenticate(username, password)
    if user:
        return jsonify({"message": "Login successful", "role": user.role}), 200
    return jsonify({"message": "Invalid credentials"}), 401

@app.route('/admin', methods=['GET'])
def admin_dashboard():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    user = authenticate(username, password)
    if user and authorize(user, 'admin'):
        return jsonify({"message": "Welcome to the admin dashboard"}), 200
    return jsonify({"message": "Unauthorized"}), 403

# Run the App
if __name__ == '__main__':
    # Start ngrok and expose the Flask app
    public_url = ngrok.connect(5000).public_url
    print(f" * Running on {public_url}")
    app.run()

ModuleNotFoundError: No module named 'flask_sqlalchemy'

ModuleNotFoundError: No module named 'flask_sqlalchemy'